In [ ]:
!pip install CatBoost
!pip install scikeras

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC

# New Imports required for the new models
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture as GMM

# Ensemble Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier # To use Keras with GridSearchCV/Scikit-learn
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

import matplotlib.pyplot as plt

Load processed data

In [ ]:
# Processed data (one-hot / True-False etc.)
train_df = pd.read_csv("train_processed.csv")
test_df = pd.read_csv("test_processed.csv")

# Assuming 'test_raw.csv' was used to get the original trip_id column
# If not available, replace 'test_raw.csv' with the actual filename containing 'trip_id'
try:
    test_raw = pd.read_csv('test_raw.csv')
except FileNotFoundError:
    print("Warning: 'test_raw.csv' not found. Assuming 'test_processed.csv' contains 'trip_id'.")
    test_raw = test_df.copy()

print(f"Train Data Shape: {train_df.shape}")
print(f"Test Data Shape: {test_df.shape}")

Train Data Shape: (12620, 82)
Test Data Shape: (5852, 81)


In [ ]:
# --- FIX: Clean column names for LightGBM compatibility ---
import re

def clean_feature_names(df):
    """
    Cleans column names by replacing problematic characters (commas, brackets,
    and other special symbols) with underscores for LightGBM compatibility.
    """
    new_cols = []
    for col in df.columns:
        # 1. Replace commas and brackets with underscores
        cleaned_col = col.replace(',', '_').replace('[', '_').replace(']', '_')
        # 2. Replace any remaining non-alphanumeric/non-underscore sequence with a single underscore
        cleaned_col = re.sub(r'[^A-Za-z0-9_]+', '_', cleaned_col)
        # 3. Clean up leading/trailing underscores and double underscores
        cleaned_col = cleaned_col.strip('_').replace('__', '_')
        new_cols.append(cleaned_col)

    df.columns = new_cols
    return df

# Apply the cleaning to both dataframes before splitting
train_df = clean_feature_names(train_df)
test_df = clean_feature_names(test_df)

print("Feature names cleaned for LightGBM compatibility.")
# -----------------------------------------------------------

Feature names cleaned for LightGBM compatibility.


Evaluate Model on Training Data

In [ ]:
def evaluate_model(model, X, y, model_name):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print(f"\n--- {model_name} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:\n", classification_report(y, y_pred, zero_division=0))

    # Optional: Display confusion matrix
    # disp = ConfusionMatrixDisplay.from_estimator(model, X, y, normalize='true')
    # disp.ax_.set_title(f"Confusion Matrix for {model_name}")
    # plt.show()

    return y_pred

Cross-Validation Split

In [ ]:
TARGET = 'spend_category'

X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

# Drop 'trip_id' from test features if it exists, as it's not a feature
if 'trip_id' in test_df.columns:
    X_test = test_df.drop(columns=['trip_id'])
else:
    X_test = test_df

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")

# Data preparation for Keras model(ANN model)
# 1. Encode the target variable (y_train) to integers (0, 1, 2...)
# This is required for One-Hot Encoding later
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
n_classes = len(le.classes_)
input_dim = X_train.shape[1] # Number of input features

# 2. Convert integer labels to one-hot vectors for Keras
y_train_keras = to_categorical(y_train_encoded, num_classes=n_classes)

print(f"Keras target prepared. Input dimension: {input_dim}, Output classes: {n_classes}")

n_classes = y_train.nunique()
print(f"Number of target classes: {n_classes}")


X_train shape: (12620, 81)
y_train shape: (12620,)
X_test shape: (5852, 81)
Keras target prepared. Input dimension: 81, Output classes: 3
Number of target classes: 3


Naive Bayes(Baseline, no tuning)

In [ ]:
nb_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('nb', GaussianNB()) # FIXED: Changed from BernoulliNB to GaussianNB
])


# Hyperparameter tuning grid for GaussianNB
nb_pipe.fit(X_train, y_train)
best_nb = nb_pipe
print("Naive Bayes (GaussianNB) fitted.")

train_pred_nb = evaluate_model(best_nb, X_train, y_train, "Gaussian Naive Bayes (Fixed)")

Naive Bayes (GaussianNB) fitted.

--- Gaussian Naive Bayes (Fixed) ---
Accuracy: 0.5826
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.82      0.82      6245
           1       0.58      0.18      0.28      4911
           2       0.27      0.90      0.42      1464

    accuracy                           0.58     12620
   macro avg       0.56      0.63      0.51     12620
weighted avg       0.66      0.58      0.56     12620


Sample predictions on test set:
Naive Bayes: [2 0 0 0 0]
submission_nb.csv saved.


Predicting Bayes

In [ ]:
test_pred_nb = best_nb.predict(X_test)

print("\nSample predictions on test set:")
print("Naive Bayes:", test_pred_nb[:5])

# SAVE SUBMISSION FILES (WITH trip_id)

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_nb = pd.DataFrame({
        "trip_id": test_raw["trip_id"],
        "spend_category": test_pred_nb.astype(int)
    })
    submission_nb.to_csv("submission_nb.csv", index=False)

Logistic Regression

In [ ]:
lr_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=42, max_iter=1000))
# 'lbfgs' is the correct solver for multinomial loss. Increased max_iter for convergence robustness.
])

# Hyperparameter tuning grid for Logistic Regression
lr_param_grid = {
    'lr__C': [0.1, 1.0, 10.0],
    'lr__fit_intercept': [True, False]
}

lr_grid_search = GridSearchCV(lr_pipe, lr_param_grid, cv=StratifiedKFold(n_splits=5), scoring='accuracy', n_jobs=-1, verbose=1)
lr_grid_search.fit(X_train, y_train)
best_lr = lr_grid_search.best_estimator_
print(f"\nBest LR Parameters: {lr_grid_search.best_params_}")

train_pred_lr = evaluate_model(best_lr, X_train, y_train, "Logistic Regression (Fixed)")



Fitting 5 folds for each of 6 candidates, totalling 30 fits


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Best LR Parameters: {'lr__C': 0.1, 'lr__fit_intercept': True}

--- Logistic Regression (Fixed) ---
Accuracy: 0.7478
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.86      0.84      6245
           1       0.67      0.73      0.70      4911
           2       0.63      0.30      0.41      1464

    accuracy                           0.75     12620
   macro avg       0.71      0.63      0.65     12620
weighted avg       0.74      0.75      0.74     12620


Sample predictions on test set:
Logistic   : [1 0 0 0 0]
submission_lr.csv saved.


Predicting Logistic Regression

In [ ]:
test_pred_lr = best_lr.predict(X_test)

print("\nSample predictions on test set:")
print("Logistic   :", test_pred_lr[:5])

# SAVE SUBMISSION FILES (WITH trip_id)

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_lr = pd.DataFrame({
        "trip_id": test_raw["trip_id"],
        "spend_category": test_pred_lr.astype(int)
    })
    submission_lr.to_csv("submission_lr.csv", index=False)
    print("submission_lr.csv saved.")

SVM

In [ ]:
# LinearSVC is significantly faster and more memory-efficient than SVC(kernel='linear').
# Note: LinearSVC does not support probability=True or the 'rbf' kernel.
lsvc_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    # dual='auto' selects the best algorithm for efficiency. max_iter increased for large data.
    ('lsvc', LinearSVC(random_state=42, dual='auto', max_iter=10000))
])

# 1. Reduce parameter search space (only C is tunable for LinearSVC)
lsvc_param_grid = {
    'lsvc__C': [0.1, 1.0, 10.0],
}

# 2. Reduce n_jobs and CV folds to mitigate memory errors
# Use n_jobs=4 (or 2) instead of -1 to limit memory consumption.
# Use cv=3 folds for faster initial tuning.
lsvc_grid_search = GridSearchCV(lsvc_pipe, lsvc_param_grid,
                                cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                                scoring='accuracy',
                                n_jobs=4, # Use a specific, limited number of cores
                                verbose=1)

lsvc_grid_search.fit(X_train, y_train)
best_svm = lsvc_grid_search.best_estimator_
print(f"\nOptimized Best SVM Parameters (LinearSVC): {lsvc_grid_search.best_params_}")

train_pred_svm = evaluate_model(best_svm, X_train, y_train, "Support Vector Machine")

Fitting 5 folds for each of 3 candidates, totalling 15 fits

Optimized Best SVM Parameters (LinearSVC): {'lsvc__C': 1.0}

--- Support Vector Machine ---
Accuracy: 0.7387
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.87      0.84      6245
           1       0.66      0.74      0.70      4911
           2       0.69      0.17      0.27      1464

    accuracy                           0.74     12620
   macro avg       0.72      0.59      0.60     12620
weighted avg       0.74      0.74      0.72     12620


Sample predictions on test set:
SVM        : [1 0 0 0 0]
submission_svm.csv saved.


Predicting SVM

In [ ]:
test_pred_svm = best_svm.predict(X_test)

print("\nSample predictions on test set:")
print("SVM        :", test_pred_svm[:5])

# SAVE SUBMISSION FILES (WITH trip_id)

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_svm = pd.DataFrame({
        "trip_id": test_raw["trip_id"],
        "spend_category": test_pred_svm.astype(int)
    })
    submission_svm.to_csv("submission_svm.csv", index=False)
    print("submission_svm.csv saved.")

Kernelized SVM (RBF)

In [ ]:
# SVC is very sensitive to NaN and scaling
rbf_svc_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    # Using the RBF kernel (the default)
    ('svc', SVC(random_state=42, kernel='rbf', probability=True))
])

# NOTE: Must use a VERY limited grid to avoid excessive runtimes (only 4 fits below)
rbf_svc_param_grid = {
    # C controls regularization, Gamma controls the influence of single training samples
    'svc__C': [1.0, 10.0],
    'svc__gamma': [0.1, 'scale'] # 'scale' uses 1 / (n_features * X.var())
}

# Use only 3 cross-validation folds and limit n_jobs (if possible)
rbf_svc_grid_search = GridSearchCV(rbf_svc_pipe, rbf_svc_param_grid,
                                   cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                                   scoring='accuracy',
                                   n_jobs=2, # Limit CPU cores
                                   verbose=1)

rbf_svc_grid_search.fit(X_train, y_train)
best_rbf_svm = rbf_svc_grid_search.best_estimator_
print(f"\nOptimized Best RBF SVM Parameters: {rbf_svc_grid_search.best_params_}")

train_pred_rbf_svm_eval = evaluate_model(best_rbf_svm, X_train, y_train, "RBF SVC Classifier")


Fitting 3 folds for each of 4 candidates, totalling 12 fits


KeyboardInterrupt: 

Predicting SVM

In [ ]:
test_pred_rbf_svm = best_rbf_svm.predict(X_test)

print("\nSample predictions on test set:")
print("RBF SVM    :", test_pred_rbf_svm[:5])

# SAVE SUBMISSION FILES (WITH trip_id)

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_rbf_svm = pd.DataFrame({
    "trip_id": test_raw["trip_id"],
    "spend_category": test_pred_rbf_svm.astype(int)
    })
    submission_rbf_svm.to_csv("submission_rbf_svm.csv", index=False)
    print("submission_rbf_svm.csv saved.")

ANN ( MLP Neural Network)

In [ ]:
# NOTE: The build_mlp_model function must be defined in the notebook prior to this cell.

# 1. Define the parameters for the best model (using your intended values)
BEST_MLP_PARAMS = {
    'units': 64,             # Neurons per layer
    'layers': 2,             # Number of hidden layers
    'learning_rate': 0.001,
}

def build_mlp_model(input_dim, output_dim, layers=2, units=64, learning_rate=0.001):
    """Creates a basic Multilayer Perceptron (ANN) model."""

    # Ensure imports are available: from keras.models import Sequential
    # from keras.layers import Dense, Dropout
    # import tensorflow as tf # for keras.optimizers.Adam

    model = Sequential()

    # Input Layer and First Hidden Layer
    # Note: input_dim should be explicitly defined for the first layer
    model.add(Dense(units=units, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.2)) # Standard regularization to prevent overfitting

    # Additional Hidden Layers
    for _ in range(layers - 1):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(0.2))

    # Output Layer (Softmax for multi-class classification)
    model.add(Dense(units=output_dim, activation='softmax'))

    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# 2. Build the Model
# Assuming input_dim and n_classes are defined in an earlier cell
best_mlp_model = build_mlp_model(
    input_dim=X_train.shape[1],
    output_dim=n_classes,
    **BEST_MLP_PARAMS
)

print("Training ANN (MLP) Model Directly...")

# 3. Train the Model Directly (Bypassing GridSearchCV)
# We use a validation split to monitor performance during training
history = best_mlp_model.fit(
    X_train,
    y_train_keras,          # Use the ONE-HOT ENCODED target
    epochs=50,
    batch_size=64,          # Using a batch size from your original grid
    validation_split=0.1,
    verbose=0 # Set to 1 if you want to see the progress
)

# --- 4. Generate Predictions and Decode ---

# Generate probabilities for the test set
y_pred_proba_mlp = best_mlp_model.predict(X_test, verbose=0)

# Convert probabilities back to integer class indices (0, 1, 2...)
y_pred_encoded = np.argmax(y_pred_proba_mlp, axis=1)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training ANN (MLP) Model Directly...
ANN (MLP) Training Complete.

Sample predictions on test set:
ANN (MLP)  : [1 0 0 1 0]
submission_mlp.csv saved.


Predicting Neural Network

In [ ]:
# Decode predictions back to original spend categories (1.0, 2.0, 3.0...)
# Assume 'le' (LabelEncoder) is available from the data prep step
test_pred_mlp = le.inverse_transform(y_pred_encoded)

# Generate train predictions for evaluation (optional, but requested)
y_train_proba_mlp = best_mlp_model.predict(X_train, verbose=0)
y_train_pred_encoded = np.argmax(y_train_proba_mlp, axis=1)
train_pred_mlp = le.inverse_transform(y_train_pred_encoded)


print("ANN (MLP) Training Complete.")
print("\nSample predictions on test set:")
print("ANN (MLP)  :", test_pred_mlp[:5])

# --- 5. SAVE SUBMISSION FILES (WITH trip_id) ---

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_mlp = pd.DataFrame({
        "trip_id": test_raw["trip_id"],
        "spend_category": test_pred_mlp.astype(int)
    })
    submission_mlp.to_csv("submission_mlp.csv", index=False)

K-Means Clustering

In [ ]:
# Used for finding natural groups in the features (X_train).
# Wrap K-Means in a pipeline to ensure data is clean and scaled
kmeans_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # Catches remaining NaNs
    ('scaler', StandardScaler()),                 # Scales features for distance calculation
    # Used for finding natural groups in the features (X_train).
    ('kmeans', KMeans(n_clusters=n_classes,
                      random_state=42,
                      n_init='auto'))
])
kmeans_pipe.fit(X_train)
kmeans_model = kmeans_pipe.named_steps['kmeans'] # Extract the fitted model for analysis
print("K-Means clustering fitted successfully.")

K-Means clustering fitted successfully.


Gaussian Mixture Model

In [ ]:
# Wrap GMM in a pipeline to ensure data is clean and scaled
gmm_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # Catches remaining NaNs
    ('scaler', StandardScaler()),                 # Scales features for proper covariance estimation
    ('gmm', GMM(n_components=n_classes,
                           random_state=42))
])

# Fit the pipeline to X_train.
gmm_pipe.fit(X_train)
gmm_model = gmm_pipe.named_steps['gmm'] # Extract the fitted model for analysis
print("GMM fitted successfully.")

GMM fitted successfully.


XGBoost

In [ ]:
xgb_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    # XGBoost does not need a scaler
    ('xgb', XGBClassifier(objective='multi:softmax',
                          num_class=n_classes,
                          use_label_encoder=False, # Suppress warning on older versions
                          eval_metric='mlogloss', # Standard multi-class metric
                          random_state=42))
])

# Using a reduced grid for faster initial results (cv=3, n_jobs=4)
xgb_param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__learning_rate': [0.05, 0.1],
    'xgb__max_depth': [3, 5]
}

xgb_grid_search = GridSearchCV(xgb_pipe, xgb_param_grid, cv=StratifiedKFold(n_splits=3), scoring='accuracy', n_jobs=4, verbose=1)
xgb_grid_search.fit(X_train, y_train)
best_xgb = xgb_grid_search.best_estimator_
print(f"\nBest XGBoost Parameters: {xgb_grid_search.best_params_}")

train_pred_xgb = evaluate_model(best_xgb, X_train, y_train, "XGBoost Classifier")
test_pred_xgb = best_xgb.predict(X_test)

print("\nSample predictions on test set:")
print("XGBoost    :", test_pred_xgb[:5])

# SAVE SUBMISSION FILES (WITH trip_id)

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_xgb = pd.DataFrame({
        "trip_id": test_raw["trip_id"],
        "spend_category": test_pred_xgb.astype(int)
    })
    submission_xgb.to_csv("submission_xgb.csv", index=False)
    print("submission_xgb.csv saved.")

LightGBM

In [ ]:
lgbm_pipe = Pipeline([
    # LightGBM does not need a scaler
    ('lgbm', LGBMClassifier(objective='multiclass',
                            num_class=n_classes,
                            metric='multi_logloss',
                            random_state=42,
                            n_jobs=-1))
])

# Using a reduced grid for faster initial results (cv=3, n_jobs=4)
lgbm_param_grid = {
    'lgbm__n_estimators': [100, 200],
    'lgbm__learning_rate': [0.05, 0.1],
    'lgbm__num_leaves': [20, 31]
}

lgbm_grid_search = GridSearchCV(lgbm_pipe, lgbm_param_grid, cv=StratifiedKFold(n_splits=3), scoring='accuracy', n_jobs=4, verbose=1)
lgbm_grid_search.fit(X_train, y_train)
best_lgbm = lgbm_grid_search.best_estimator_
print(f"\nBest LightGBM Parameters: {lgbm_grid_search.best_params_}")

train_pred_lgbm = evaluate_model(best_lgbm, X_train, y_train, "LightGBM Classifier")
test_pred_lgbm = best_lgbm.predict(X_test)

print("\nSample predictions on test set:")
print("LightGBM   :", test_pred_lgbm[:5])

# SAVE SUBMISSION FILES (WITH trip_id)

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_lgbm = pd.DataFrame({
        "trip_id": test_raw["trip_id"],
        "spend_category": test_pred_lgbm.astype(int)
    })
    submission_lgbm.to_csv("submission_lgbm.csv", index=False)
    print("submission_lgbm.csv saved.")

CatBoost

In [ ]:
# --- CATBOOST-SPECIFIC DATA PREPARATION (Using Raw Files) ---
import re

# 1. Re-define helper functions used in original preprocessing (crucial for consistency)
def map_range_to_midpoint(range_str):
    try:
        if '-' in range_str:
            start, end = map(float, re.findall(r'\d+', range_str))
            return (start + end) / 2
        else: # Handle single number values
            return float(range_str)
    except:
        return np.nan # Use NaN for CatBoost to impute

age_mapping = {'<18': 10, '18-24': 21, '25-44': 35, '45-64': 55, '65+': 70}

# 2. Load Raw Data
raw_train_df = pd.read_csv("train.csv")
raw_test_df = pd.read_csv("test.csv")
TARGET = 'spend_category'

# 3. Clean Target and Store Test ID
raw_train_df.dropna(subset=[TARGET], inplace=True)
y_train_cbt = raw_train_df[TARGET].astype(int)

# 4. Apply Feature Engineering (Ordinal Mappings)
for df in [raw_train_df, raw_test_df]:
    # Apply Midpoint mapping
    df['days_booked_midpoint'] = df['days_booked_before_trip'].apply(map_range_to_midpoint)
    df['total_trip_days_midpoint'] = df['total_trip_days'].apply(map_range_to_midpoint)
    # Apply Age Ordinal mapping
    df['age_group_ordinal'] = df['age_group'].map(age_mapping).fillna(np.nan)

    # Drop original ordinal columns
    df.drop(columns=['days_booked_before_trip', 'total_trip_days', 'age_group'], inplace=True)

# 5. Separate X and Identify Categorical Features
X_train_cbt = raw_train_df.drop(columns=[TARGET, 'trip_id'])
X_test_cbt = raw_test_df.drop(columns=['trip_id'])

# CatBoost needs the list of *string* column names that are categorical
CBT_CAT_FEATURES = X_train_cbt.select_dtypes(include=['object']).columns.tolist()

print(f"\nCatBoost Data Ready.")
print(f"Number of string categorical features identified: {len(CBT_CAT_FEATURES)}")
# --------------------------------------------------------------------------------


cbt_pipe = Pipeline([
    ('cbt', CatBoostClassifier(verbose=0,
                                random_state=42,
                                loss_function='MultiClass',
                                eval_metric='MultiClass',
                                # Pass the categorical feature names to CatBoost
                                cat_features=CBT_CAT_FEATURES, # <-- THIS IS THE KEY CHANGE
                                thread_count=4))
])



#Hyperparameter tuning grid for CatBoost
cbt_param_grid = {
    'cbt__n_estimators': [100, 200],
    'cbt__learning_rate': [0.05, 0.1]
}

cbt_grid_search = GridSearchCV(cbt_pipe, cbt_param_grid, cv=StratifiedKFold(n_splits=3), scoring='accuracy', n_jobs=4, verbose=1)

# Fit CatBoost with the specially prepared raw data
cbt_grid_search.fit(X_train_cbt, y_train_cbt) # <-- USE CBT VARIABLES
best_cbt = cbt_grid_search.best_estimator_
print(f"\nBest CatBoost Parameters: {cbt_grid_search.best_params_}")

train_pred_cbt = evaluate_model(best_cbt, X_train_cbt, y_train_cbt, "CatBoost Classifier")
test_pred_cbt = best_cbt.predict(X_test_cbt)

print("\nSample predictions on test set:")
# NOTE: CatBoost multi-class prediction returns a 2D array, so we flatten it
test_pred_cbt_flat = test_pred_cbt.flatten()

print("CatBoost   :", test_pred_cbt_flat[:5])

# SAVE SUBMISSION FILES (WITH trip_id)

# Ensure test_raw has trip_id column
if 'trip_id' not in test_raw.columns:
    print("Error: 'trip_id' column not found in test_raw/test_df! Cannot generate submission file.")
else:
    submission_cbt = pd.DataFrame({
    # Use the raw_test_id created during CatBoost prep
    "trip_id": test_raw["trip_id"],
    "spend_category": test_pred_cbt_flat.astype(int)
    })
    submission_cbt.to_csv("submission_cbt.csv", index=False)
    print("submission_cbt.csv saved.")